<a href="https://colab.research.google.com/github/ningjingzhiyuan507/AnalysisTool/blob/master/d2l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 配置GPU版本pytorch

In [ ]:
!pip install torch==1.8.1
!pip install torchvision==0.9.1

In [11]:
!pip install d2l==0.17.3

     |████████████████████████████████| 82 kB 619 kB/s 
     |████████████████████████████████| 11.6 MB 11.2 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 61 kB 7.1 MB/s 
     |████████████████████████████████| 9.9 MB 29.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the p

In [3]:
# 测试GPU
import torch
from torch import nn

In [ ]:
!nvidia-smi

In [4]:
torch.cuda.device_count()  # 仅有一个GPU

1

In [10]:
X = torch.rand(2, 3, device='cuda:0')

RuntimeError: ignored

Mon Mar 14 02:49:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    57W / 149W |    516MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
X

tensor([[0.4178, 0.9268, 0.5924],
        [0.9275, 0.0595, 0.6194]], device='cuda:0')

# LeNet-5

In [12]:
import torch
from torch import nn
from d2l import torch as d2l

In [13]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [14]:
batchSize = 256
trainIter, testIter = d2l.load_data_fashion_mnist(batch_size=batchSize)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
def evaluateAccuracyGpu(net, dataIter, device=None):
  """使用GPU计算模型在数据集上的精度"""
  if isinstance(net, nn.Module):
    net.eval()  # 设置为评估模式
    if not device:
      device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    with torch.no_grad():
      for X, y in dataIter:
        if isinstance(X, list):
          X = [x.to(device) for x in X]
        else:
          X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [16]:
def train_ch6(net, trainIter, testIter, numEpochs, lr, device):
  """用GPU训练模型"""
  def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
      nn.init.xavier_uniform_(m.weight)  # 使用Xavier随机初始化模型参数
  net.apply(init_weights)
  print("training on", device)
  net.to(device)
  optimizer = torch.optim.SGD(net.parameters(), lr=lr)
  loss = nn.CrossEntropyLoss()
  animator = d2l.Animator(xlabel='epoch', xlim=[1, numEpochs], legend=['train loss', 'train acc', 'test acc'])
  timer, numBatches = d2l.Timer(), len(trainIter)
  for epoch in range(numEpochs):
    # 训练损失之和，训练准确率之和，样本数
    metric = d2l.Accumulator(3)
    net.train()
    for i, (X, y) in enumerate(trainIter):
      timer.start()
      optimizer.zero_grad()
      X, y = X.to(device), y.to(device)
      yHat = net(X)
      l = loss(yHat, y)
      l.backward()
      optimizer.step()
      with torch.no_grad():
        metric.add(l * X.shape[0], d2l.accuracy(yHat, y), X.shape[0])
      timer.stop()
      trainL = metric[0] / metric[2]
      trainAcc = metric[1] / metric[2]
      if (i+1) % (numBatches // 5) == 0 or i == numBatches - 1:
        animator.add(epoch + (i+1) / numBatches, (trainL, trainAcc, None))
    testAcc = evaluateAccuracyGpu(net, testIter)
    animator.add(epoch + 1, (None, None, testIter))
  print(f'loss {trainL: .3f}, train acc {trainAcc:.3f},'
        f'test acc {testAcc:.3f}')
  print(f'{metric[2] * numEpochs / timer.sum():.1f} examples/sec '
        f'on {str(device)}')

In [17]:
lr, numEpochs = 0.9, 10
train_ch6(net, trainIter, testIter, numEpochs, lr, d2l.try_gpu())

TypeError: ignored

ImportError: ignored

<Figure size 252x180 with 1 Axes>